In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.misc import imread, imresize

from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, Dense, Flatten, Dropout, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import ELU,Lambda
from sklearn.model_selection import train_test_split
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Dropout, Flatten, Lambda, ELU
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.convolutional import Convolution2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2


def resize_image(image_path):
    path = image_path.replace(' ', '')
    img = imread(path)
    
    row,col,channels=img.shape
    top=int(row*0.35)
    bottom = int(row*0.875)
    img = img[top:bottom,:]
    #img = imresize(img, (32, 32))
    img = cv2.resize(img, (220,66), interpolation=cv2.INTER_AREA)
    img = add_random_shadow(img)
    return img

def add_random_shadow(image):
    top_y = 320*np.random.uniform()
    top_x = 0
    bot_x = 160
    bot_y = 320*np.random.uniform()
    image_hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    shadow_mask = 0*image_hls[:,:,1]
    X_m = np.mgrid[0:image.shape[0],0:image.shape[1]][0]
    Y_m = np.mgrid[0:image.shape[0],0:image.shape[1]][1]
    shadow_mask[((X_m-top_x)*(bot_y-top_y) -(bot_x - top_x)*(Y_m-top_y) >=0)]=1
    #random_bright = .25+.7*np.random.uniform()
    if np.random.randint(2)==1:
        random_bright = .5
        cond1 = shadow_mask==1
        cond0 = shadow_mask==0
        if np.random.randint(2)==1:
            image_hls[:,:,1][cond1] = image_hls[:,:,1][cond1]*random_bright
        else:
            image_hls[:,:,1][cond0] = image_hls[:,:,1][cond0]*random_bright    
    image = cv2.cvtColor(image_hls,cv2.COLOR_HLS2RGB)
    return image

# def trans_image(image,steer,trans_range):
#     # Translation
#     tr_x = trans_range*np.random.uniform()-trans_range/2
#     steer_ang = steer + tr_x/trans_range*2*.2
#     tr_y = 40*np.random.uniform()-40/2
#     #tr_y = 0
#     Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
#     image_tr = cv2.warpAffine(image,Trans_M,(220,66))
    
#     return image_tr,steer_ang

def get_model():

	model = Sequential()
	#adam = Adam(lr=0.001)

	# Nvidia
	model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(66, 220, 3), output_shape=(66, 220, 3)))
	model.add(Convolution2D(24, 5, 5, subsample=(2, 2), border_mode="valid", init='he_normal'))
	model.add(ELU())
	model.add(Convolution2D(36, 5, 5, subsample=(2, 2), border_mode="valid", init='he_normal'))
	model.add(ELU())
	#model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', dim_ordering='default'))
	model.add(Dropout(0.3))
	model.add(Convolution2D(48, 5, 5, subsample=(2, 2), border_mode="valid", init='he_normal'))
	model.add(ELU())
	model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid", init='he_normal'))
	model.add(Dropout(0.2))
	model.add(ELU())   
	model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid", init='he_normal'))
	#model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), border_mode='valid', dim_ordering='default'))
	model.add(Flatten())
	model.add(ELU())
	model.add(Dense(1164, init='he_normal'))
	model.add(ELU())
	model.add(Dense(100, init='he_normal'))
	model.add(ELU())
	model.add(Dense(50, init='he_normal'))
	model.add(ELU())
	model.add(Dense(10, init='he_normal'))
	model.add(ELU())
	model.add(Dropout(0.2))
	model.add(Dense(1, init='he_normal'))

	# Use the Adam optimizer to optimize the mean squared error
	#model.compile(optimizer=adam, loss="mse")	

	return model

# def a_model():
#     model = Sequential([
#         BatchNormalization(input_shape=(32, 32, 3)),
#         Conv2D(32, 3, 3, activation='relu', border_mode='same'),
#         Conv2D(32, 3, 3, activation='relu'),
#         MaxPooling2D(pool_size=(2, 2)),
#         Dropout(0.25),
#         Conv2D(64, 3, 3, activation='relu', border_mode='same'),
#         Conv2D(64, 3, 3, activation='relu'),
#         MaxPooling2D(pool_size=(2, 2)),
#         Dropout(0.25),
#         Flatten(),
#         Dense(512, activation='relu'),
#         Dropout(0.5),
#         Dense(1)
#     ])
#     return model

# load the data
# driving_log = pd.read_csv('driving_log.csv')
# image_paths = pd.concat([driving_log['center'], driving_log['left'], driving_log['right']])
# image_paths = np.array(image_paths, dtype=pd.Series)
# mirror_paths = driving_log['center']
# mirror_paths = np.array(mirror_paths, dtype=pd.Series)
# angles = pd.concat([driving_log['steering'], driving_log['steering'] + 0.08, driving_log['steering'] - 0.08,
#                    -driving_log['steering']])
# angles = np.array(angles, dtype=pd.Series)

driving_log = pd.read_csv('driving_log.csv')
image_paths = pd.concat([driving_log['center'], driving_log['left'], driving_log['right']])
image_paths = np.array(image_paths, dtype=pd.Series)
mirror_paths = driving_log['center']
mirror_paths = np.array(mirror_paths, dtype=pd.Series)

angles = pd.concat([driving_log['steering'], driving_log['steering'] + 0.2, driving_log['steering'] - 0.2,
                   -driving_log['steering']])
angles = np.array(angles, dtype=pd.Series)


# preprocess images
images = [resize_image(path) for path in image_paths]
images.extend([np.fliplr(resize_image(path)) for path in mirror_paths])
#images.extend([resize_image(path) for path in mirror_paths1])
images = np.array(images)
angles = np.array([np.asarray([angle], np.float32) for angle in angles])

#images,angles=trans_image(images,angles,0.3)

images_training, images_validation, angles_training, angles_validation = train_test_split(images, angles, test_size=0.2,
                                                                                          random_state=42)
nb_training = images_training.shape[0]
nb_validation = images_validation.shape[0]
nb_epoch = 30

#  train the model
#my_model = a_model()
my_model= get_model()
my_model.summary()
my_model.compile(optimizer=Adam(lr=0.0001), loss='mse')
generator = ImageDataGenerator(width_shift_range=0.2, fill_mode='nearest',shear_range=0.2)
#generator.fit(images_training)

# only save the best weights
checkpoint = ModelCheckpoint('model29.h5', monitor='val_loss', verbose=1, save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, verbose=1)
callbacks = [early_stop, checkpoint]
my_model.fit_generator(generator.flow(images_training, angles_training, batch_size=128), nb_training, nb_epoch,
                       validation_data=generator.flow(images_validation, angles_validation, batch_size=128),
                       callbacks=callbacks, nb_val_samples=nb_validation)

# save the model
with open("model29.json", "w") as json_file:
    json_file.write(my_model.to_json())

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 66, 220, 3)    0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 31, 108, 24)   1824        lambda_1[0][0]                   
____________________________________________________________________________________________________
elu_1 (ELU)                      (None, 31, 108, 24)   0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 14, 52, 36)    21636       elu_1[0][0]                      
___________________________________________________________________________________________

In [ ]:
# def trans_image(image,steer,trans_range):
#     # Translation
#     tr_x = trans_range*np.random.uniform()-trans_range/2
#     steer_ang = steer + tr_x/trans_range*2*.2
#     tr_y = 40*np.random.uniform()-40/2
#     #tr_y = 0
#     Trans_M = np.float32([[1,0,tr_x],[0,1,tr_y]])
#     image_tr = cv2.warpAffine(image,Trans_M,(cols,rows))
    
#     return image_tr,steer_ang

In [9]:
# driving_log = pd.read_csv('driving_log.csv')
# image_paths = pd.concat([driving_log['center'], driving_log['left'], driving_log['right']])
# image_paths = np.array(image_paths, dtype=pd.Series)
# mirror_paths = driving_log['center']
# mirror_paths = np.array(mirror_paths, dtype=pd.Series)
# mirror_paths1 = driving_log['right']
# mirror_paths1 = np.array(mirror_paths1, dtype=pd.Series)
# angles = pd.concat([driving_log['steering'], driving_log['steering'] + 0.08, driving_log['steering'] - 0.08,
#                    -driving_log['steering'],driving_log['steering'] - 0.1])
# angles = np.array(angles, dtype=pd.Series)

# print (angles.shape)

(40180,)


In [ ]:
# nvidia model:
# Epoch 20/20
# 25600/25715 [============================>.] - ETA: 0s - loss: 0.0223Epoch 00019: val_loss did not improve
# 25715/25715 [==============================] - 222s - loss: 0.0223 - val_loss: 0.0208

In [ ]:
# images = [resize_image(path) for path in image_paths]
# images.extend([np.fliplr(resize_image(path)) for path in mirror_paths])
# images.extend([resize_image(path) for path in mirror_paths1])
# images = np.array(images)
# angles = np.array([np.asarray([angle], np.float32) for angle in angles])

In [1]:
#  print(images.shape)#(32144, 66, 220, 3)

NameError: name 'images' is not defined

In [ ]:

# images,angles=trans_image(images,angles,0.3)

In [ ]:
#  print(images.shape)#(32144, 66, 220, 3)